In [9]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()


#carregar tabela ( atenção ao caminho)
arquivo_json_grande = spark.read.json('s3a://stc-grupo04-henrique-sptech-raw-data/casos_obitos_doencas_pre_multiline.json')
arquivo_csv = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-raw-data/part-00000-1dc7c2db-2925-4e86-8dd8-3445327415e3.c000.csv')
arquivo_txt = spark.read.option('delimiter', '|').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-raw-data/data_IDH_pipe.txt')

print(arquivo_json_grande.count())
print(arquivo_csv.count())
print(arquivo_txt.count())

print( type(arquivo_json_grande) )
print( type(arquivo_csv) )
print( type(arquivo_txt) )


1000001


1000000
28
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
#exibir a tabela
arquivo_json_grande.printSchema()
arquivo_csv.printSchema()
arquivo_txt.printSchema()


root
 |-- _corrupt_record: string (nullable = true)
 |-- asma: string (nullable = true)
 |-- cardiopatia: string (nullable = true)
 |-- codigo_ibge: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- data_inicio_sintomas: string (nullable = true)
 |-- diabetes: string (nullable = true)
 |-- diagnostico_covid19: string (nullable = true)
 |-- doenca_hematologica: string (nullable = true)
 |-- doenca_hepatica: string (nullable = true)
 |-- doenca_neurologica: string (nullable = true)
 |-- doenca_renal: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- imunodepressao: string (nullable = true)
 |-- obesidade: string (nullable = true)
 |-- obito: string (nullable = true)
 |-- outros_fatores_de_risco: string (nullable = true)
 |-- pneumopatia: string (nullable = true)
 |-- puerpera: string (nullable = true)
 |-- sindrome_de_down: string (nullable = true)

root
 |-- source_id: string (nullable = true)
 |-- dataNotificacao

In [11]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
# removendo as colunas que não são necessarias do json
json_pre_tratado = arquivo_json_grande.select(
    col('estado').alias('ESTADO'),
    col('cs_sexo').alias('SEXO'),
    col('idade').cast(IntegerType()).alias('IDADE'),
    
    col('asma').alias('ASMA'),
    col('cardiopatia').alias('CARDIOPATIA'),
    col('diabetes').alias('DIABETE'),
    col('obesidade').alias('OBESIDADE'),
    col('sindrome_de_down').alias('SIN_DOWN')
)
# removendo as colunas que não são necessarias do csv

csv_pre_tratado = arquivo_csv.select(
    col('estadoIBGE').alias('ESTADO'),
    col('sexo').alias('SEXO'),
    col('idade').cast(IntegerType()).alias('IDADE'),
    
    col('racaCor').alias('ETNIA'),
    col('evolucaoCaso').alias('EVOLUCAO_CASO'),
    col('codigoLaboratorioPrimeiraDose').alias('PRIMEIRA_DOSE'),
    col('codigoLaboratorioSegundaDose').alias('SEGUNDA_DOSE')
)

txt_pre_tratado=arquivo_txt.select(
    col('Territorialidades').alias('TERRITORIALIDADES'),
    col('Média de anos de estudo 2017').alias('MEDIA_ANOS_ESTUDOS'),
    col('IDHM Educação 2017').alias('INDICE_EDUCACAO'),
    col('IDHM Longevidade 2017').alias('LONGEVIDADE'),
    col('IDHM Renda 2017').alias('RENDA'),
    col('IDHM 2017').alias('IDH'),
)

json_pre_tratado.printSchema()

csv_pre_tratado.printSchema()

txt_pre_tratado.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)

root
 |-- TERRITORIALIDADES: string (nullable = true)
 |-- MEDIA_ANOS_ESTUDOS: string (nullable = true)
 |-- INDICE_EDUCACAO: string (nullable = true)
 |-- LONGEVIDADE: string (nullable = true)
 |-- RENDA: string (nullable = true)
 |-- IDH: string (nullable = true)



In [13]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
print("salvando no s3")
json_pre_tratado.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedJson.csv')
csv_pre_tratado.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedCsv.csv')
txt_pre_tratado.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedTxt.csv')

print("salvo")

salvando no s3


22/05/30 01:54:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:24 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 01:54:34 WA

salvo


In [ ]:

#fazendo calculos
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType
json_pre_tratado.select(sum(col('idade').cast(IntegerType())).alias('soma')).show()
csv_pre_tratado.select(sum(col('idade').cast(IntegerType())).alias('soma')).show()


In [5]:
# mostrando average de idade por estado
json_pre_tratado.groupby('estado').avg().show()
csv_pre_tratado.groupby('estado').avg().show()

+------+------------------+
|estado|        avg(IDADE)|
+------+------------------+
|    SC|39.196452933151434|
|    RO|  39.4694171578258|
|    PI|39.044805321854824|
|    AM| 39.22893088987189|
|    RR| 39.21560846560847|
|    GO|39.171869199878515|
|  null|              null|
|    TO| 39.42488331388565|
|    MT| 39.07632927397351|
|    SP| 39.19261509583087|
|    ES| 39.20011461318052|
|    PB|39.147176115542145|
|    RS|39.090246248557136|
|    MS|  39.4780073308897|
|    AL|39.308251473477405|
|    MG| 39.29136937574106|
|    PA| 39.21754769263562|
|    BA|39.085467218852074|
|    SE| 39.41999612102405|
|    PE|39.196951219512194|
+------+------------------+
only showing top 20 rows



+------+------------------+
|estado|        avg(IDADE)|
+------+------------------+
|    SC| 35.85145317545748|
|    RO|35.515151515151516|
|    PI| 37.00729927007299|
|    AM| 35.82213438735178|
|    RR|23.545454545454547|
|    GO| 35.25589836660617|
|  null|38.992098935073855|
|    TO| 37.05714285714286|
|    MT| 35.66554054054054|
|    SP|36.169973792978105|
|    PB|37.172284644194754|
|    ES|  36.7906976744186|
|    RS| 37.74193548387097|
|    MS| 35.38107416879795|
|    AL| 32.14035087719298|
|    MG| 36.77768595041322|
|    PA| 38.37238493723849|
|    BA|  36.4173140954495|
|    SE| 37.36974789915966|
|    PE|  37.7910284463895|
+------+------------------+
only showing top 20 rows



In [6]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

json_pre_tratado.printSchema()
print("inserindo azure")
json_pre_tratado.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('overwrite') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://yuiti.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'teste2') \
.option('user', 'yuiti') \
.option('password', 'Thiago123') \
.save()
print("Salvo")

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)

inserindo azure


Salvo
